In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from pytorch3d.io import load_objs_as_meshes
from pytorch3d.utils import ico_sphere
from pytorch3d.structures import Meshes, Pointclouds
from pytorch3d.ops import sample_points_from_meshes, packed_to_padded#,knn_points, estimate_pointcloud_normals, knn_gather

import trimesh

from ops.utils import *
from ops.mesh_geometry import *

import pyvista as pv
pv.start_xvfb()
pv.set_jupyter_backend('html')

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Set the device
if torch.cuda.is_available():
    device = torch.device("cuda:1")
else:
    device = torch.device("cpu")

For Mesh

In [ ]:
trg_mesh = load_objs_as_meshes(["data_example/Holesman.obj","data_example/Genus2.obj","data_example/Genus6.obj"], device=device)
trg_mesh = normalize_mesh(trg_mesh, rescalar=0.9)
points_np = trg_mesh[0].verts_packed().cpu().numpy()
faces_np = trg_mesh[0].faces_packed().cpu().numpy()
trg_trmesh = trimesh.Trimesh(vertices=points_np, faces=faces_np)

In [ ]:
print("====== Mesh Gaussian Curvature & Euler Characteristic ======")

Euler_gt = []
for mesh_tem in trg_mesh:
    trimesh_tem = trimesh.Trimesh(vertices=mesh_tem.verts_packed().cpu().numpy(), faces=mesh_tem.faces_packed().cpu().numpy())
    Euler_gt.append(trimesh_tem.euler_number)
print('Euler characteristic Ground Truth (Combinational):', Euler_gt)
print('Genus Ground Truth:', 1 - np.array(Euler_gt)/2)
print('-----------------------------------------------')


gscurv_padded = vert_feature_packed_padded(trg_mesh, get_gaussian_curvature_vertices_packed(trg_mesh).view(-1,1))
dual_areas_padded = vert_feature_packed_padded(trg_mesh, get_dual_area_vertex_packed(trg_mesh).view(-1,1))
diff_euler = (gscurv_padded*dual_areas_padded).sum(dim=1).view(-1)/2/np.pi
diff_genus = 1 - diff_euler/2
print('Differentiable Genus (local gauss-bonnet):', diff_genus.cpu().numpy())


curvature_vertex_packed = get_gaussian_curvature_vertices_from_face_packed(trg_mesh).view(-1,1)
curvature_vertex_padded = vert_feature_packed_padded(trg_mesh, curvature_vertex_packed.view(-1,1))
dual_areas_padded = vert_feature_packed_padded(trg_mesh, get_dual_area_vertex_packed(trg_mesh).view(-1,1))
diff_euler = (curvature_vertex_padded*dual_areas_padded).sum(dim=1).view(-1)/2/np.pi
diff_genus = 1 - diff_euler/2
print('Differentiable Genus (gathered face curvature):', diff_genus.cpu().numpy())


gc_face_packed = get_gaussian_curvature_faces_packed(trg_mesh).view(-1,1)
gc_face_padded = face_feature_packed_padded(trg_mesh, gc_face_packed.view(-1,1))
area_face_padded = face_feature_packed_padded(trg_mesh, trg_mesh.faces_areas_packed().view(-1,1))
diff_euler = (gc_face_padded*area_face_padded).sum(dim=1).view(-1)/2/np.pi
diff_genus = 1 - diff_euler/2
print('Differentiable Genus (face curvature):', diff_genus.cpu().numpy())

In [ ]:
print("====== Mean Curvature ======")
mean_curvature = get_mean_curvature_vertices_packed(trg_mesh).view(-1,1)
mean_curvature_padded = vert_feature_packed_padded(trg_mesh, mean_curvature)


In [ ]:
print("====== Total Curvature ======")

total_curvature_padded = ((mean_curvature_padded*2)**2 - 2*curvature_vertex_padded)
total_curvature_padded = total_curvature_padded.clamp(min=0)

In [ ]:
# Visualize Curvatures
pl = pv.Plotter(notebook=1, shape=(1, 3))
## subplot 1
pl.subplot(0, 0)
index = 0
out_trimesh = trimesh.Trimesh(trg_mesh[index].verts_packed().cpu().numpy(), trg_mesh[index].faces_packed().cpu().numpy())
curvature_color = get_gaussian_curvature_vertices_packed(trg_mesh[index]).detach().cpu()
curvature_color = torch.tanh(curvature_color/6).numpy()
pl.add_mesh(out_trimesh, scalars=curvature_color, cmap="viridis", show_scalar_bar=True, clim=[-1, 1],
            scalar_bar_args={'title': 'Gaussian Curvature (tanh)', 'position_x': 0.25, 'position_y': 0.88})
pl.zoom_camera(1.5)

## subplot 2
pl.subplot(0, 1)
index = 0
out_trimesh = trimesh.Trimesh(trg_mesh[index].verts_packed().cpu().numpy(), trg_mesh[index].faces_packed().cpu().numpy())
curvature_color = get_mean_curvature_vertices_packed(trg_mesh[index]).detach().cpu()
curvature_color = torch.tanh(curvature_color/6).numpy()
pl.add_mesh(out_trimesh, scalars=curvature_color, cmap="magma", show_scalar_bar=True, clim=[-1, 1],
            scalar_bar_args={'title': 'Mean Curvature (tanh)', 'position_x': 0.25, 'position_y': 0.88})
pl.zoom_camera(1.5)

## subplot 3
pl.subplot(0, 2)
index = 0
out_trimesh = trimesh.Trimesh(trg_mesh[index].verts_packed().cpu().numpy(), trg_mesh[index].faces_packed().cpu().numpy())
curvature_color = total_curvature_padded[index].detach().cpu()
curvature_color = torch.tanh(curvature_color/600).numpy()
pl.add_mesh(out_trimesh, scalars=curvature_color, cmap="cividis", show_scalar_bar=True, clim=[0, 1],
            scalar_bar_args={'title': 'Total Curvature (tanh)', 'position_x': 0.25, 'position_y': 0.88})
pl.zoom_camera(1.5)
pl.show()
pl.screenshot("results/mesh_curvature_example.png", window_size=[1000*2, 450*2])
pl.close()

For PCL

In [ ]:
from ops.pcl_geometry import *

In [ ]:
trg_mesh = load_objs_as_meshes(["data_example/Kitten.obj", "data_example/Genus2.obj"], device=device)

trg_mesh = normalize_mesh(trg_mesh)

trg_trmesh = trimesh.Trimesh(vertices=trg_mesh[0].verts_packed().cpu().numpy(), faces=trg_mesh[0].faces_packed().cpu().numpy())


In [ ]:
num_samples = np.random.randint(1000,20000)
pointscloud, normals_gt = sample_points_from_meshes(trg_mesh, num_samples, return_normals=True)

global_mean = pointscloud.mean(dim=-2,keepdim=True)
global_cov = torch.matmul((pointscloud - global_mean).transpose(-1,-2), (pointscloud - global_mean))/(num_samples-1)
global_square_trace = global_cov.diagonal(offset=0, dim1=-2, dim2=-1).sum(-1)
print('Normalized by scale of global_square_trace:', global_square_trace.cpu().numpy())

trg_mesh = trg_mesh.update_padded((trg_mesh.verts_padded() - global_mean)/torch.sqrt(global_square_trace).unsqueeze(-1).unsqueeze(-1))
pointscloud, normals_gt = sample_points_from_meshes(trg_mesh, 50000, return_normals=True)


In [ ]:
pcl_with_frames = Differentiable_Global_Geometry_PointCloud(pointscloud, k = 8)


face_area_padded = face_feature_packed_padded(trg_mesh, trg_mesh.faces_areas_packed())
# pointscloud, normals = sample_points_from_meshes(trg_mesh, 1000, return_normals=True)
print("====== Local Voronoi Area ======")
print("Area GT:", face_area_padded.sum(dim=-1).cpu().numpy())


with torch.no_grad():

    weingarten_fields = pcl_with_frames.weingarten_fields()
    ##### determinant of the Weingarten map
    gaussian_curvature = weingarten_fields.det().unsqueeze(-1)

    ##### trace of the Weingarten map
    mean_curvature = weingarten_fields.diagonal(offset=0, dim1=-1, dim2=-2).mean(-1).unsqueeze(-1)

    ##### norm of the Weingarten map
    total_curvature = (weingarten_fields@weingarten_fields.transpose(-1,-2)).diagonal(offset=0, dim1=-1, dim2=-2).sum(-1).unsqueeze(-1)

    area = pcl_with_frames.local_voronoi_area(local_W=64)

print("Area Estimated:", area.sum(dim=1).view(-1).cpu().numpy())
print("====== Gaussian Curvature and Euler Characteristic ======")
print("Diff Euler Number:", (gaussian_curvature*area).sum(dim=1).view(-1).cpu().numpy()/2/np.pi)
print("Diff Genus:", 1 - (gaussian_curvature*area).sum(dim=1).view(-1).cpu().numpy()/4/np.pi)






In [ ]:
### Plot the point cloud with the estimated gaussian curvature
pl = pv.Plotter(notebook=True, shape=(1, 3))
# index = np.random.randint(0,pointscloud.shape[0])
index = 0
pl.subplot(0, 0)
rescaled_gs_curv = gaussian_curvature[index: index+1]
rescaled_gs_curv = torch.tanh(rescaled_gs_curv/6)
pl.add_points(pcl_with_frames.pointscloud.detach().cpu().numpy()[index], scalars=rescaled_gs_curv.detach().cpu().numpy(), 
              point_size=5,cmap="viridis", clim=[-1,1], show_scalar_bar=True,
              scalar_bar_args={'title': 'Gaussian Curvature (tanh)', 'position_x': 0.25, 'position_y': 0.88})
pl.zoom_camera(1.5)
pl.subplot(0, 1)
rescaled_mean_curv = mean_curvature[index: index+1]
rescaled_mean_curv = torch.tanh(rescaled_mean_curv/6)
pl.add_points(pcl_with_frames.pointscloud.detach().cpu().numpy()[index], scalars=rescaled_mean_curv.detach().cpu().numpy(), 
              show_scalar_bar=True,
              point_size=5,cmap="magma", clim=[-1,1], scalar_bar_args={'title': 'Mean Curvature (tanh)', 'position_x': 0.25, 'position_y': 0.88})
pl.zoom_camera(1.5)

pl.subplot(0, 2)
rescaled_total_curv = total_curvature[index: index+1]
rescaled_total_curv = torch.tanh(rescaled_total_curv/100)
pl.add_points(pcl_with_frames.pointscloud.detach().cpu().numpy()[index], scalars=rescaled_total_curv.detach().cpu().numpy(), 
              show_scalar_bar=True,
              point_size=5,cmap="cividis", clim=[0,1], scalar_bar_args={'title': 'Total Curvature (log)', 'position_x': 0.25, 'position_y': 0.88})
pl.zoom_camera(1.5)
pl.show()
pl.screenshot("results/pointcloud_curvature_example.png", window_size=[1200*2, 450*2])
pl.close()